# Predicting Tax Value for Single Family Homes

by Braeden Wright
03-30-2022

## Project Goal

- Find the key drivers of property value for single family properties and construct an ML Regression model that will predict property tax assessed values (tax_value) for those homes.

## Project Description

- TBD

## Initial Questions

- Does more square footage always lead to a larger tax value?
- Would the year the house was built positively or negatively affect tax value?
- TBD

## Data Dictionary



# Imports

In [1]:
# Foundation
import pandas as pd
import numpy as np
from scipy import math

# Visuals
import matplotlib.pyplot as plt
import seaborn as sns


# Metric and Modeling Tools
import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing
from sklearn.linear_model import LogisticRegression

# Custom
import wrangle_prep as wp
import explore as ep


# Required Data
df = wp.final_wrangle_zillow()
df.head(3)

Reading cleaned data from csv file...


,bedrooms,bathrooms,sqr_feet,tax_value,year_built,fips
0,4.0,2.0,3633.0,296425.0,2005,6037
1,3.0,2.0,2077.0,646760.0,1926,6037
2,3.0,1.0,1244.0,169471.0,1950,6037
